In [48]:
from bs4 import BeautifulSoup
import requests
import subprocess
import pandas as pd

# Get list

In [49]:
url_alesp = 'https://www.al.sp.gov.br/sigp-portal/default.xhtml'

In [50]:
search = requests.get(url_alesp)
soup = BeautifulSoup(search.content, 'html.parser')

In [51]:
id_name = {}

for option in soup.find_all('option'):
    id_name[option['value']] = option.text

In [52]:
id_name
del id_name['']

In [53]:
id_name_final = {}
    
date_list = []

for key, value in id_name.items():
    try:
        key = int(key)
        id_name_final[key] = value
    except:
        date_list.append(key)

In [ ]:
id_name_final

{1137: 'Adalberto Freitas',
 1138: 'Adriana Borgo',
 1139: 'Agente Federal Danilo Balas',
 1185: 'Alessandra Monteiro ',
 1140: 'Alex de Madureira',
 496: 'Alexandre Pereira',
 1178: 'Altair Moraes',
 363: 'Analice Fernandes',
 429: 'André do Prado',
 1177: 'Aprigio',
 1176: 'Arthur do Val',
 1175: 'Ataide Teruel',
 124: 'Barros Munhoz',
 367: 'Beth Sahão',
 1174: 'Bruno Ganem',
 492: 'Caio França',
 153: 'Campos Machado',
 1173: 'Carla Morando',
 431: 'Carlão Pignatari',
 433: 'Carlos Cezar',
 417: 'Carlos Giannazi',
 1172: 'Castello Branco',
 435: 'Cauê Macris',
 1171: 'Cezar',
 141: 'Conte Lopes',
 1170: 'Coronel Nishikawa',
 469: 'Coronel Telhada',
 1169: 'Daniel José',
 1168: 'Daniel Soares',
 1167: 'Delegada Graciela',
 1166: 'Delegado Bruno Lima',
 493: 'Delegado Olim',
 1183: 'Dirceu Dalben',
 1165: 'Douglas Garcia',
 1164: 'Dr. Jorge Do Carmo',
 1179: 'Dra. Damaris Moura',
 386: 'Ed Thomas',
 228: 'Edmir Chedid',
 254: 'Edna Macedo',
 327: 'Emidio de Souza',
 372: 'Enio Tatto'

In [ ]:
def get_data(name_id, year_month):

    headers = {
        'Connection': 'keep-alive',
        'Cache-Control': 'max-age=0',
        'Origin': 'https://www.al.sp.gov.br',
        'Upgrade-Insecure-Requests': '1',
        'DNT': '1',
        'Content-Type': 'application/x-www-form-urlencoded',
        'Cookie': 'JSESSIONID=DADA756F83E36702D14EE6374783FFAF; BNES_JSESSIONID=32aUEBofyGSws/W0V3Il1uTuT88ErNcrTKSZwN7GzvZQeBw/0PIEhyGZ6TkKbhLV472YPNlkLn5mQBgGCfifnGgorgF7G+p+'
    }

    data = {
        'formFiltros': 'formFiltros',
        'formFiltros:listaDeputados': name_id,
        'formFiltros:mesAnoSessao': year_month,
        'formFiltros:j_idt17': 'Filtrar',
        'javax.faces.ViewState': '2704732828298455370:8076758869967991483'
    }

    result = requests.post('https://www.al.sp.gov.br/sigp-portal/default.xhtml', 
                           headers=headers, 
                           data=data)

    result_soup = BeautifulSoup(result.content, 'html.parser')
    table_content = result_soup.find_all('td', {'rowspan' : '3'})
    attendance_list = []

    for a in table_content:
        attendance_list.append(a.text)
            
    df = pd.DataFrame(attendance_list, columns=['presenca_raw'])
    df['nome_id'] = name_id
    df['mes_ano'] = year_month
    
    return df

# Test for one

In [ ]:
dfops = get_data('1172', '9/2019')
dfops.head()

,presenca_raw,nome_id,mes_ano
0,\n\n\n02\n\n\n\nO\n\n\n\nCQR\n\n\n\n,1172,9/2019
1,\n\n\n03\n\n\n\nO\n\n\n\nP\n\n\n\n,1172,9/2019
2,\n\n\n03\n\n\n\nE\n\n\n\nP\n\n\n\n,1172,9/2019
3,\n\n\n04\n\n\n\nO\n\n\n\nP\n\n\n\n,1172,9/2019
4,\n\n\n04\n\n\n\nE\n\n\n\nP\n\n\n\n,1172,9/2019


# Run for all of them

In [ ]:
df_final = pd.DataFrame()

In [ ]:
for k,v in id_name_final.items():
    for d in date_list:
        df = get_data(k, d)
        df['nome'] = v
        df_final = df_final.append(df)